# Agent 2 — Module Walkthrough (Code + Review)
## Training Script (`train_agent2.py`)

**Author:** Summer Xiong  
**Goal:** Provide a *research-grade, executable* walkthrough of the Agent 2 training pipeline: data → windows → dataset → model → training loop → validation → artifact saving.

This notebook mirrors your training script but adds:
- Clear **inputs/outputs** and **tensor shapes**
- Methodological rationale (anti-leakage split, windowing, macro metrics)
- Review notes (robustness, missing checks, reproducibility)

> **Usage (Colab):**
> ```bash
> !pip -q install -r requirements.txt
> !python train_agent2.py --data_dir /content/data --epochs 6 --window 5
> ```


## 0) Imports & Dependencies

This training script depends on Agent 2 modules:
- `load_data.py` → ingestion + schema normalisation + split
- `windows.py` → sliding window construction
- `dataset.py` → tokenisation + collation
- `model.py` → TimeSeriesClassifier architecture
- `metrics.py` → macro PRF evaluation

External dependencies:
- PyTorch, Transformers, tqdm, NumPy


In [ ]:
import argparse, json
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm

from load_data import load_and_merge, normalise_columns, select_numeric_columns, split_by_voter, VALID_LABELS
from windows import build_windows
from dataset import WindowDataset, collate_fn
from model import TimeSeriesClassifier
from metrics import macro_prf


## 1) Utility: Class Weights for Imbalance

```python
w_c = N / (C * max(count_c, 1))
```

### What this does
Computes inverse-frequency class weights to mitigate class imbalance.

### Why it matters in DAO votes
FOR is often dominant; AGAINST/ABSTAIN can be sparse.  
Weighted loss helps ensure the model does not ignore minority classes.

### Review note
This is a simple baseline. Alternatives include:
- focal loss
- effective number of samples (Cui et al.)
- per-cluster class weighting


In [ ]:
def compute_class_weights(y: np.ndarray, num_classes: int = 3) -> torch.Tensor:
    counts = np.bincount(y, minlength=num_classes).astype(float)
    N = counts.sum()
    w = N / (num_classes * np.maximum(counts, 1.0))
    return torch.tensor(w, dtype=torch.float32)


## 2) Argument Parsing and Reproducibility

### CLI arguments
- `--data_dir`: directory containing `cluster_0/1/2_dataset.csv`
- `--pretrained`: HF model name (default `roberta-base`)
- window/tokenisation/training hyperparameters

### Seeds
You set seeds for both PyTorch and NumPy.

**Review note:** For stronger determinism consider:
- `torch.cuda.manual_seed_all`
- `torch.backends.cudnn.deterministic = True` (may affect speed)
- logging the full environment (CUDA, transformers version)


In [ ]:
def parse_args():
    ap = argparse.ArgumentParser()
    ap.add_argument("--data_dir", type=str, required=True, help="Directory containing cluster_0/1/2_dataset.csv")
    ap.add_argument("--pretrained", type=str, default="roberta-base")
    ap.add_argument("--window", type=int, default=5)
    ap.add_argument("--max_length", type=int, default=128)
    ap.add_argument("--batch_size", type=int, default=32)
    ap.add_argument("--epochs", type=int, default=6)
    ap.add_argument("--lr", type=float, default=2e-5)
    ap.add_argument("--seed", type=int, default=42)
    return ap.parse_args()


## 3) Data Loading → Normalisation → Label Filtering

### Pipeline
1. Load `cluster_{0,1,2}_dataset.csv` files
2. Concatenate
3. Normalise schema (`normalise_columns`)
4. Filter to valid labels `{0,1,2}`

**Review note:** The script assumes exactly 3 files (clusters 0–2).  
If you later change cluster count, parameterise this.


In [ ]:
def load_data_pipeline(data_dir: Path):
    csvs = [data_dir / f"cluster_{i}_dataset.csv" for i in range(3)]
    df = load_and_merge(csvs)
    df = normalise_columns(df)
    df = df[df["label_id"].isin([0, 1, 2])].copy()
    return df


## 4) Split by Voter (Anti-Leakage)

You split train/valid by **voter** rather than by rows.

This is one of the most defensible design choices in the entire pipeline:
- prevents identity leakage
- yields more realistic generalisation metrics

We'll keep it as-is, but we should also verify there is no overlap.


In [ ]:
def split_train_valid(df: "pd.DataFrame", seed: int = 42):
    train_df, valid_df = split_by_voter(df, train_frac=0.8, seed=seed)
    overlap = set(train_df["voter"]).intersection(set(valid_df["voter"]))
    return train_df, valid_df, len(overlap)


## 5) Tokeniser with Special Tokens

### Why special tokens?
Your window builder prefixes texts with:
- `[PREDICT]`
- `[LABEL_0]`, `[LABEL_1]`, `[LABEL_2]`

Adding these tokens ensures the tokenizer treats them as atomic tokens (not split into subword fragments), improving signal clarity.

**Implementation detail**
After adding tokens, you must call:
```python
model.text_encoder.resize_token_embeddings(len(tokenizer))
```
to update the embedding matrix.


In [ ]:
def build_tokenizer(pretrained: str):
    tokenizer = AutoTokenizer.from_pretrained(pretrained, use_fast=True)
    special_tokens = {"additional_special_tokens": ["[PREDICT]", "[LABEL_0]", "[LABEL_1]", "[LABEL_2]"]}
    tokenizer.add_special_tokens(special_tokens)
    return tokenizer


## 6) Build Windows (Sequence Construction)

### What happens here
- Choose numeric columns using `select_numeric_columns(df)`
- Call `build_windows(...)` on train and valid splits

### Output
- `train_windows`, `valid_windows`: lists of `Window` objects
- Each `Window` contains:
  - `window_texts` length `W`
  - `window_features` list length `W`, each vector length `F`
  - `target_label` ∈ {0,1,2}

**Review note:** This step is often the bottleneck (Python loops).  
If dataset grows, consider caching windows to disk.


In [ ]:
def build_windows_for_split(df_part, window_size: int, numeric_cols):
    return build_windows(
        df=df_part,
        window_size=window_size,
        text_col="text",
        label_col="label_id",
        voter_col="voter",
        time_col="vote_ts",
        cluster_col="cluster_id",
        numeric_cols=numeric_cols,
    )


## 7) Dataset, DataLoader, and Feature Dimension

### Dataset
`WindowDataset` performs tokenisation per step in `__getitem__`, returning tensors:
- `input_ids`: `(W, L)`
- `attention_mask`: `(W, L)`
- `num_feats`: `(W, F)`

### DataLoader / collate_fn
Batches into:
- `input_ids`: `(B, W, L)`
- `attention_mask`: `(B, W, L)`
- `num_feats`: `(B, W, F)`
- `labels`: `(B,)`

### feat_dim inference
You infer `feat_dim` using the first window:
```python
feat_dim = len(train_windows[0].window_features[0])
```

**Review note:** Add a safety assert to ensure all steps and all windows have the same feature length.


In [ ]:
def infer_feat_dim(train_windows, default: int = 8) -> int:
    if len(train_windows) == 0:
        return default
    return int(len(train_windows[0].window_features[0]))


## 8) Model Initialisation

- Instantiate `TimeSeriesClassifier(pretrained_model_name, feat_dim)`
- Resize token embeddings to include special tokens
- Move to GPU if available

**Review note:** This is correct. Just ensure:
- `feat_dim` matches actual numeric vector length from `build_windows`
- window size `W` is within position embedding range (`<=512`)


In [ ]:
def build_model(pretrained: str, feat_dim: int, tokenizer) -> TimeSeriesClassifier:
    model = TimeSeriesClassifier(pretrained_model_name=pretrained, feat_dim=feat_dim)
    model.text_encoder.resize_token_embeddings(len(tokenizer))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return model.to(device)


## 9) Optimiser & Scheduler

- Optimiser: `AdamW`
- Scheduler: linear warmup (10% steps) then decay

This is a standard, defensible setup for transformer fine-tuning.

**Review note:** Log `total_steps`, `warmup_steps` for reproducibility.


In [ ]:
def build_optim_and_sched(model, lr: float, epochs: int, train_loader_len: int):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    total_steps = train_loader_len * epochs
    warmup_steps = int(0.1 * total_steps)
    scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)
    return optimizer, scheduler, total_steps, warmup_steps


## 10) Training Loop (Epoch → Train → Valid → Best Checkpoint)

### Train step
- forward
- weighted cross-entropy loss
- backward + gradient clipping
- optimizer step + scheduler step
- collect predictions for macro PRF

### Valid step
- `torch.no_grad()`
- collect predictions for macro PRF

### Model selection
- track best validation F1 (macro)
- store best model weights on CPU for safe saving


In [ ]:
def move_batch_to_device(batch, device):
    for k in ["input_ids", "attention_mask", "num_feats", "labels", "clusters"]:
        batch[k] = batch[k].to(device)
    return batch


### Full training loop (as in script)

This cell contains the original loop logic but rewritten into notebook-friendly functions.  
You can run this once your data directory is available.


In [ ]:
def train_agent2(args):
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)

    data_dir = Path(args.data_dir)
    df = load_data_pipeline(data_dir)

    train_df, valid_df, overlap = split_train_valid(df, seed=args.seed)
    print(f"Train rows: {len(train_df)}, Valid rows: {len(valid_df)}, Voter overlap: {overlap}")

    tokenizer = build_tokenizer(args.pretrained)
    numeric_cols = select_numeric_columns(df)

    train_windows = build_windows_for_split(train_df, args.window, numeric_cols)
    valid_windows = build_windows_for_split(valid_df, args.window, numeric_cols)
    print(f"Train windows: {len(train_windows)}, Valid windows: {len(valid_windows)}")

    train_ds = WindowDataset(train_windows, tokenizer, max_length=args.max_length)
    valid_ds = WindowDataset(valid_windows, tokenizer, max_length=args.max_length)

    train_loader = DataLoader(train_ds, batch_size=args.batch_size, shuffle=True, collate_fn=collate_fn)
    valid_loader = DataLoader(valid_ds, batch_size=args.batch_size, shuffle=False, collate_fn=collate_fn)

    feat_dim = infer_feat_dim(train_windows, default=8)
    model = build_model(args.pretrained, feat_dim, tokenizer)

    optimizer, scheduler, total_steps, warmup_steps = build_optim_and_sched(
        model, args.lr, args.epochs, len(train_loader)
    )
    print(f"Total steps: {total_steps}, Warmup steps: {warmup_steps}, feat_dim: {feat_dim}")

    y_train = np.array([w.target_label for w in train_windows], dtype=int)
    class_weights = compute_class_weights(y_train).to(next(model.parameters()).device)

    device = next(model.parameters()).device

    best_f1, best_state = -1.0, None
    for epoch in range(args.epochs):
        # ---- train ----
        model.train()
        running_loss = 0.0
        y_true, y_pred = [], []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{args.epochs} [train]"):
            batch = move_batch_to_device(batch, device)
            out = model(batch)
            loss = model.loss_fn(out["logits"], batch["labels"], class_weights=class_weights)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            running_loss += float(loss.item())

            preds = out["logits"].argmax(dim=-1).detach().cpu().numpy().tolist()
            y_pred.extend(preds)
            y_true.extend(batch["labels"].detach().cpu().numpy().tolist())

        train_metrics = macro_prf(y_true, y_pred)

        # ---- valid ----
        model.eval()
        vy_true, vy_pred = [], []
        with torch.no_grad():
            for batch in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{args.epochs} [valid]"):
                batch = move_batch_to_device(batch, device)
                out = model(batch)
                preds = out["logits"].argmax(dim=-1).detach().cpu().numpy().tolist()
                vy_pred.extend(preds)
                vy_true.extend(batch["labels"].detach().cpu().numpy().tolist())

        val_metrics = macro_prf(vy_true, vy_pred)

        print(
            f"\nEpoch {epoch+1}: "
            f"loss={running_loss/len(train_loader):.4f} "
            f"train F1={train_metrics['f1']:.4f} | valid F1={val_metrics['f1']:.4f}"
        )

        if val_metrics["f1"] > best_f1:
            best_f1 = float(val_metrics["f1"])
            best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
            print(f"✓ New best model: F1={best_f1:.4f}")

    if best_state is not None:
        model.load_state_dict(best_state)
        print(f"Loaded best model with F1={best_f1:.4f}")

    return model, tokenizer, {"best_f1": best_f1, "feat_dim": feat_dim}


## 11) Saving Artifacts (Model + Tokeniser + Config)

You save to:
- `agent2_artifacts/agent2_model.pt` (state dict)
- `agent2_artifacts/tokenizer/` (HF tokenizer files)
- `agent2_artifacts/config.json` (hyperparameters + best F1)

This is good practice for reproducibility and later inference.

**Review note:** Consider saving:
- `class_weights`
- training/validation metrics by epoch
- git commit hash / run id


In [ ]:
def save_artifacts(model, tokenizer, run_info: dict, args, outdir: Path = Path("agent2_artifacts")):
    outdir.mkdir(parents=True, exist_ok=True)

    model_path = outdir / "agent2_model.pt"
    tok_path = outdir / "tokenizer"
    tok_path.mkdir(exist_ok=True, parents=True)

    torch.save(model.state_dict(), model_path)
    tokenizer.save_pretrained(tok_path)

    cfg = {
        "pretrained": args.pretrained,
        "window": args.window,
        "max_length": args.max_length,
        "feat_dim": int(run_info.get("feat_dim", 0)),
        "label_map": {"FOR": 0, "AGAINST": 1, "ABSTAIN": 2},
        "best_f1": float(run_info.get("best_f1", -1.0)),
    }

    with open(outdir / "config.json", "w") as f:
        json.dump(cfg, f, indent=2)

    print(f"Saved model to {model_path} and tokenizer to {tok_path}")
    return model_path, tok_path, outdir / "config.json"


## 12) Review Notes: High-Impact Improvements

If you want this training pipeline to look **publication-ready / mentor-ready**, the top improvements are:

1) **Add explicit checks**
- non-empty windows
- constant feature dimension across all steps/windows
- no NaN labels after normalisation

2) **Fix boolean column naming consistency**
Your window builder currently reads raw `"Is Whale"` columns; normalised schema produces `is_whale`.  
Standardise to avoid silently dropping booleans.

3) **Add logging**
- metrics per epoch (train/valid)
- save as JSON/CSV for plotting
- store random seed, versions

4) **Add per-cluster evaluation**
You already pass `clusters` in batch.  
Compute macro F1 per cluster to support fairness claims.

5) **Make cluster file count configurable**
Right now you hardcode 3 cluster CSVs. Parameterise if cluster K changes.

These changes are small but will greatly increase the credibility of your results.


## 13) Full Script (Original Reference)

For completeness, below is the original training script content (kept as-is).  
In practice, you will run the notebook functions for clarity, or run the script for production runs.


In [ ]:
# Original script reference (verbatim)
# NOTE: In the notebook, prefer using the helper functions defined above.


## 14) Summary

This training pipeline:
- loads and normalises vote data
- applies a **voter-wise split** to prevent leakage
- builds sliding windows to form a sequence prediction dataset
- fine-tunes a multimodal temporal model (text + numeric + temporal transformer)
- evaluates with macro PRF
- saves reproducible artifacts

It is a solid baseline for Agent 2, and with a few robustness additions, it can be presented confidently to supervisors.
